# <span style="color:purple">*Getting the data*</span>

In [81]:
# Visit the following link to download the dataset in your local machine: 
# https://raw.githubusercontent.com/blast-analytics-marketing/RFM-analysis/master/sample-orders.csv

import pandas as pd
orders = pd.read_csv('sample-data.csv',sep=',')

In [82]:
# Exploring the dataset
orders.head()

,order_date,order_id,customer,grand_total
0,9/7/2011,CA-2011-100006,Dennis Kane,378.0
1,7/8/2011,CA-2011-100090,Ed Braxton,699.0
2,3/14/2011,CA-2011-100293,Neil Franz?sisch,91.0
3,1/29/2011,CA-2011-100328,Jasper Cacioppo,4.0
4,4/8/2011,CA-2011-100363,Jim Mitchum,21.0


# <span style="color:purple">*Preparing the data*</span>

### <span style="color:orange">*Since the last order date in our dataset is Dec 31 2014, that is the date we will use to calculate Recency.*</span>

### <span style="color:orange">*Set this date to the current day and extract all orders until yesterday.*</span>


In [83]:
# You have a Text object. The strftime function requires a datetime object.
# The code below takes an intermediate step of converting your Text to a datetime using strptime.

import datetime
testeddate = '2014/12/31'
NOW = datetime.datetime.strptime(testeddate,'%Y/%m/%d')

In [84]:
# Convert the date_placed column into datetime

orders['order_date'] = pd.to_datetime(orders['order_date'])


# <span style="color:purple">*Create the RFM Table*</span>


In [85]:
rfmTable = orders.groupby('customer').agg({'order_date': lambda x: (NOW - x.max()), # Recency
                                        'order_id': lambda x: len(x),               # Frequency
                                        'grand_total': lambda x: x.sum()})          # Monetary Value

rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)



In [86]:
# Converting the time delta to days instead of including the term 'days' in the actual column.

rfmTable['recency'] = rfmTable['recency'].astype('timedelta64[D]')

In [87]:
# Checking the results

rfmTable.head()

,frequency,recency,monetary_value
customer,,,
Aaron Bergman,3,415.0,887.0
Aaron Hawkins,7,12.0,1744.0
Aaron Smayling,7,88.0,3050.0
Adam Bellavance,8,54.0,7756.0
Adam Hart,10,34.0,3249.0


# <span style="color:purple">*Validating the RFM Table*</span>


### <span style="color:orange">*Looking at the table above, we notice that 'Aaron Bergman' (one of our hypothetical clients) has the following charecteristics: *</span>

* frequency = 3
* monetary value = $887
* recency = 415 days


### <span style="color:orange">*Hence, we would like to check if that is true indeed. *</span>

 

In [88]:
aaron = orders[orders['customer']=='Aaron Bergman']
aaron

,order_date,order_id,customer,grand_total
624,2011-02-19,CA-2011-152905,Aaron Bergman,13.0
665,2011-03-07,CA-2011-156587,Aaron Bergman,310.0
2336,2013-11-11,CA-2013-140935,Aaron Bergman,564.0


In [89]:
(NOW - datetime.datetime(2013,11,11)).days==415

True

# <span style="color:purple">*Determining RFM Quartiles*</span>



In [90]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,frequency,recency,monetary_value
0.25,5.0,30.0,1145.0
0.50,6.0,75.0,2257.0
0.75,8.0,183.0,3784.0


# <span style="color:purple">*Send quantiles to a dictionary, easier to use.*</span>


In [91]:
quantiles = quantiles.to_dict()
quantiles

{'frequency': {0.25: 5.0, 0.5: 6.0, 0.75: 8.0},
 'monetary_value': {0.25: 1145.0, 0.5: 2257.0, 0.75: 3784.0},
 'recency': {0.25: 30.0, 0.5: 75.0, 0.75: 183.0}}

# <span style="color:purple">*Creating the RFM segmentation table*</span>


In [92]:
rfmSegmentation = rfmTable

### <span style="color:orange">*We create two classes for the RFM segmentation since, being high recency is bad, while high frequency and monetary value is good.*</span>


In [93]:
# Arguments (x = value, p = recency, d = monetary_value, frequency, k = quartiles dict)

def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
# --------------------------------------------------------------------------------------------------------------------------#    


# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)

def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [94]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [95]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [96]:
rfmSegmentation.head()

,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,3,415.0,887.0,4,4,4,444
Aaron Hawkins,7,12.0,1744.0,1,2,3,123
Aaron Smayling,7,88.0,3050.0,3,2,2,322
Adam Bellavance,8,54.0,7756.0,2,2,1,221
Adam Hart,10,34.0,3249.0,2,1,2,212


### <span style="color:orange">* Uncomment any of the following lines to: *</span>

1. copy data to clipboard
2. save it to a CSV file.


In [97]:
# rfmSegmentation.to_clipboard()
#rfmSegmentation.to_csv('rfm-table.csv', sep=',')

### <span style="color:orange">* Who are the top 5 best customers? by RFM Class (111), high spenders who buy recently and frequently? *</span>



In [98]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort('monetary_value', ascending=False).head(5)

C:\Users\diozel\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Sanjit Engle,11,9.0,12210.0,1,1,1,111
John Lee,11,21.0,9801.0,1,1,1,111
Pete Kriz,12,9.0,8647.0,1,1,1,111
Harry Marie,10,2.0,8237.0,1,1,1,111
Lena Creighton,12,16.0,7661.0,1,1,1,111


# <span style="color:purple">* Getting the total score for each client and adding the column "Total Score" to the dataframe *</span>

In [99]:
rfmSegmentation['Total Score'] = rfmSegmentation['R_Quartile'] + rfmSegmentation['F_Quartile'] +rfmSegmentation['M_Quartile']

In [100]:
rfmSegmentation.head()

,frequency,recency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass,Total Score
customer,,,,,,,,
Aaron Bergman,3,415.0,887.0,4,4,4,444,12
Aaron Hawkins,7,12.0,1744.0,1,2,3,123,6
Aaron Smayling,7,88.0,3050.0,3,2,2,322,7
Adam Bellavance,8,54.0,7756.0,2,2,1,221,5
Adam Hart,10,34.0,3249.0,2,1,2,212,5


### <span style="color:orange">* Assigning a specific label to each client, according to the total score column. So:*</span>

* If total_Score = 12, then "Excellent"
* If 7 <= total_Score < 12, then "Good"
* If 3 <= total_Score < 7, then "Bad"
* If 1 <= total_Score < 3, then "Only 1 transaction?"




In [101]:
# Setting up the label for each client and adding the column "Label" to the dataframe

label = [0] * len(rfmSegmentation)

for i in range(0,len(rfmSegmentation)):

    if rfmSegmentation['Total Score'][i] == 12:
        label[i] = "Excellent"
        
    elif rfmSegmentation['Total Score'][i] >= 7 :
        label[i] = "Good"
        
    elif rfmSegmentation['Total Score'][i] >= 3:
        label[i] = "Bad"
        
    else:
        label[i] = "Only 1 transaction?"        

In [102]:
# Adding the 'Label' column to our dataframe

rfmSegmentation['Label'] = label

In [103]:
# Count the frequency that a value occurs in a dataframe column for the labels.

rfmSegmentation['Label'].value_counts()

Good         437
Bad          289
Excellent     67
Name: Label, dtype: int64

### <span style="color:orange">* We are changing the background color of the label, based on the label value so that we can sort the clients faster.*</span>



In [104]:

def color(val):
    if val == "Excellent":
        color = 'green'
    elif val == "Good":
        color = 'yellow'
    elif val == "Bad":
        color = 'red'
    return 'background-color: %s' % color

rfmSegmentation.style.applymap(color, subset=['Label'])

### <span style="color:orange">* Exporting the dataset into a csv file.*</span>

In [105]:
rfmSegmentation.to_csv('rfm-table-data.csv', sep=',')